In [1]:
import os
import sys
#from tempfile import TemporaryDirectory

import matplotlib as mpl
import numpy as np
import pandas as pd

import flopy

print(sys.version)
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {mpl.__version__}")
print(f"flopy version: {flopy.__version__}")

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.24.3
matplotlib version: 3.7.2
flopy version: 3.7.0.dev0


In [2]:
# temporary directory

#temp_dir = TemporaryDirectory()

#model_ws = temp_dir.name

model_ws = '.\\' # Se establece el directorio de trabajo del modelo (model_ws)

# define la ruta completa del modelo (modelpth) combinando el directorio de trabajo (model_ws) 
# con el nombre de una subcarpeta específica ("p01SpSt_4mf6_mt")
modelpth = os.path.join(model_ws, "p01SpSt_4mf6_mt")

print(modelpth)

#modelname = "p01SpSt_4mf6_mt" #Define el nombre del modelo (modelname)
modelname = "p01SpatialStresses" #Define el nombre del modelo (modelname)
#mfexe = "mfnwt"
mtexe = "mt3d-usgs_1.1.0_64.exe" # Especifica la ruta del ejecutable de MT3D-USGS (mtexe) 


.\p01SpSt_4mf6_mt


In [3]:
# Start by setting up MT3D-USGS class and use files created with MF6-GWF for setting up a number of the BTN arrays
# mt es una instancia de Mt3dms, que representa un modelo de MT3DMS.
# modelname=modelname asigna el nombre del modelo definido anteriormente.
# model_ws=modelpth establece el directorio donde el modelo guardará y leerá archivos.
# version="mt3d-usgs" especifica la versión del software MT3D-USGS.
# namefile_ext="nam" define la extensión del archivo de nombre como .nam.
# exe_name="mt3d-usgs_1.1.0_64" indica el nombre del ejecutable de MT3D-USGS a usar.
# ftlfree=True habilita el uso de un archivo de enlace de flujo libre de formato (FTL).

mt = flopy.mt3d.Mt3dms(
    modelname=modelname,
    model_ws=modelpth,
    version="mt3d-usgs",
    namefile_ext="nam",
    exe_name="mt3d-usgs_1.1.0_64",
   
    ftlfree=True
)


In [4]:
# Se definen las dimensiones del modelo: 6 capas (nlay), 1 fila (nrow) y 101 columnas (ncol).
# sconc es un arreglo que inicializa las concentraciones iniciales en cero y establece 1.0 en la primera columna de cada capa.
# icbund es un arreglo que indica las celdas activas para el transporte. Las celdas de la primera columna de cada capa se definen como celdas de concentración constante (-1).
# Se crea una instancia btn de Mt3dBtn, que es el paquete básico de transporte para MT3DMS. Se configura con:
# Las dimensiones del modelo (nlay, nrow, ncol).
# Tres periodos de estrés (nper).
# Una especie química (ncomp) y una especie móvil (mcomp).
# Un arreglo que define la conductividad hidráulica de cada capa (laycon).
# Distancia entre columnas (delr) y filas (delc).
# Altura inicial (htop).
# Espesor de cada capa (dz).
# Duración de cada periodo de estrés (perlen), con un solo paso de tiempo (nstp) y un multiplicador de paso de tiempo (tsmult).
# Porosidad (prsity).
# Concentraciones iniciales (sconc) y condiciones de frontera (icbund).
# El número de unidad del archivo (unitnumber) y el formato de impresión de la concentración (ifmtcn).

nlay=6
nrow=1
ncol=101
sconc = np.zeros((nlay,nrow,ncol),dtype=float)
sconc[:,0,0]=1.0E+03
#print(sconc)
icbund = np.ones((nlay,nrow,ncol),dtype=int)
icbund[:,0,0]=-1 #primer columna de cada capa
#print(icbund)
dz = np.ones((nlay,nrow,ncol),dtype=float)
dz=dz * 10.0
#print(dz)
btn = flopy.mt3d.Mt3dBtn( mt,
        nlay=6,
        nrow=1,
        ncol=101,
        nper=3,
        ncomp=1, # Num.total especies quimicas 
        mcomp=1, # Num.especies móviles             
        laycon= [1,1,1,1,1,1],
        delr=10,
        delc=1,
        htop=60,
        dz=dz,
        perlen=200,
        nstp=10,
        dt0=50,
        tsmult=1,
        prsity=.25,
        sconc = sconc,
        unitnumber=31,
        ifmtcn=1,
        icbund = icbund
)

In [5]:
# mixelm se establece en 0, lo que indica que se utiliza el esquema de advección estándar de diferencias finitas.
# percel es el tamaño de la celda de contorno utilizado en el cálculo del transporte advectivo, configurado en 0.5.
# mxpart define el número máximo de partículas permitidas en la simulación, establecido en 800,000.
# nadvfd se configura en 1, lo que selecciona el esquema de ponderación aguas arriba para la advección.
# Se crea una instancia adv de Mt3dAdv, el paquete de advección para MT3DMS, configurado con los parámetros anteriores.

mixelm = 0
percel = 0.50000
mxpart = 800000
nadvfd = 1  # (1 = Upstream weighting)

adv = flopy.mt3d.Mt3dAdv(
    mt, mixelm=mixelm, percel=percel, mxpart=mxpart, nadvfd=nadvfd
)

In [6]:
# Descripción breve del código que configura el paquete de Dispersión (DSP) en MT3DMS utilizando FloPy:
# al es la dispersividad longitudinal, configurada en 10 metros.
# trpt es el ratio de la dispersividad transversal horizontal respecto a la dispersividad longitudinal al, configurado en 0.1.
# trpv es el ratio de la dispersividad transversal vertical respecto a la dispersividad longitudinal al, configurado en 0.01.
# dmcoef es el coeficiente de difusión molecular, establecido en 1.0000×10−10
# Se crea una instancia dsp del paquete de Dispersión Mt3dDsp para MT3DMS, con soporte para múltiples mecanismos de difusión (multiDiff=True).

al = 10  # longitudinal dispersivity
trpt = 0.1  # ratio of the horizontal transverse dispersivity to 'AL'
trpv = 0.01  # ratio of the vertical transverse dispersitvity to 'AL'
dmcoef = 1.0000e-9

dsp = flopy.mt3d.Mt3dDsp(
    mt, al=al, trpt=trpt, trpv=trpv, dmcoef=dmcoef, multiDiff=False
)

In [7]:
# Descripción del paquete GCG (Generalized Conjugate Gradient Solver) en MT3DMS utilizando FloPy:
# mxiter es el número máximo de iteraciones totales del solver, configurado en 10,000.
# iter1 es el número máximo de iteraciones internas por paso temporal, configurado en 50.
# isolve especifica el método de solución, con el valor 3 indicando un solver de gradiente conjugado ortogonalizado (Orthomin).
# ncrs es un parámetro que controla la estrategia de relajación cruzada, aquí establecido en 0 (sin relajación cruzada).
# accl es el factor de aceleración para la convergencia del solver, establecido en 1.0.
# cclose es el criterio de convergencia para el solver, configurado en 1.00×10−6.
# iprgcg es la frecuencia de impresión de la información del solver, aquí establecido en 0 (sin impresión).

mxiter = 10000
iter1 = 50
isolve = 3
ncrs = 0
accl = 1.000000
cclose = 1.00e-06
iprgcg = 0

gcg = flopy.mt3d.Mt3dGcg(
    mt,
    mxiter=mxiter,
    iter1=iter1,
    isolve=isolve,
    ncrs=ncrs,
    accl=accl,
    cclose=cclose,
    iprgcg=iprgcg,
)

In [8]:
# Definición de Tipos de Fuentes: 
# Se obtiene y muestra el diccionario itype de FloPy para MT3DMS, que asocia tipos de fuentes a valores numéricos.
# Estructura de Datos Predeterminada: 
# Se imprime la estructura de datos predeterminada para el paquete SSM, que define cómo se estructuran 
# los datos de entrada.
# Inicialización de Datos SSM: Se crea un diccionario ssm_data para almacenar los datos de fuentes y sumideros para varios periodos de estrés.
# Datos para Periodo de Estrés 0: Se agregan datos específicos de fuentes para el primer periodo de estrés, incluyendo un pozo (WEL) y modificaciones en concentraciones.
# Repeticiones de Datos: Se utilizan bucles para añadir múltiples entradas a ssm_data para cada periodo de estrés, cambiando concentraciones y tipos de fuente.
# Creación del Paquete SSM: Se instancia el paquete SSM con el modelo MT3DMS, pasando ssm_data y especificando un máximo de fuentes/sumideros.
# Escritura del Archivo SSM: Se escribe el archivo de configuración del paquete SSM basado en los datos proporcionados.


In [9]:
# Esta es la celda original para construir ssm_data
itype = flopy.mt3d.Mt3dSsm.itype_dict()
print(itype)
print(flopy.mt3d.Mt3dSsm.get_default_dtype())

flota=float(0.0)
flota1 = float(-1.0)

ssm_data = {}

ssm_data[0] = [(4, 0, 49, flota, itype['WEL'])]
for j in range(ncol):
    ssm_data[0].append((0, 0, j, 1.5, 7))
for j in range(ncol):
    ssm_data[0].append((0, 0, j, flota, 8))
    
ssm_data[1] = [(4, 0, 49, 0.0, itype['WEL'])]
for j in range(ncol):
    ssm_data[1].append((0, 0, j, 2.5, 7))
for j in range(ncol):
    ssm_data[1].append((0, 0, j, -1.5, 8))

ssm_data[2] = [(4, 0, 49, 0.0, itype['WEL'])]
for j in range(ncol):
    ssm_data[2].append((0, 0, j, flota, 7))
for j in range(ncol):
    ssm_data[2].append((0, 0, j, flota1, 8))
        
#for k in range(nlay):
#    for i in range(nrow):     
#        ssm_data[0].append((k, i, 0, 0.0, itype["GHB"], 0.0, 0.0))

ssm = flopy.mt3d.Mt3dSsm(
    model = mt,
    dtype=None,
    stress_period_data=ssm_data,
    mxss = 215 )
#ssm.write_file()

{'CHD': 1, 'BAS6': 1, 'PBC': 1, 'WEL': 2, 'DRN': 3, 'RIV': 4, 'GHB': 5, 'MAS': 15, 'CC': -1}
[('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('css', '<f4'), ('itype', '<i4')]


In [10]:
## 1. Versión personalizada para crear el archivo .ssm
class CustomMt3dSsm(flopy.mt3d.Mt3dSsm):
    def write_file(self, f=None):
        # Si no se especifica archivo, usar el predeterminado
        if f is None:
            f = self.fn_path

        # Abrir el archivo para escritura
        with open(f, 'w') as file:
            # Escribir encabezado con las banderas de opciones
            file.write(" T F F F F F F F F F\n")
            file.write(f"{self.mxss:>10}\n")  # Número máximo de celdas (mxss)

            # Iterar sobre los datos del periodo de estrés y escribirlos
            for stress_period, data in self.stress_period_data.data.items():
                # Corregir el número de entradas (n_entries) del periodo de estrés
                n_entries = len(data)
                file.write(f"{max(203, n_entries):>10}         0                                  # stress period {stress_period + 1}\n")

                # Iterar sobre cada entrada en los datos del periodo de estrés
                for idx, entry in enumerate(data):
                    k, i, j, concentration, itype = entry
                    # Aplicar el formato entero a la columna 4 para cada periodo estrés
                    if stress_period in [0, 1, 2] and idx == 0:
                        concentration_str = f"{int(concentration):>10}"
                    else:
                        concentration_str = f"{concentration:10.1f}"

                    formatted_entry = (
                        f"{k + 1:>10}",  # Índice k (capa), ajustado para empezar desde 1
                        f"{i + 1:>10}",  # Índice i (fila), ajustado para empezar desde 1
                        f"{j + 1:>10}",  # Índice j (columna), ajustado para empezar desde 1
                        concentration_str,  # Concentración con o sin decimal
                        f"{itype:>10}"  # Tipo de fuente/sumidero (itype)
                    )
                    line = "".join(formatted_entry)
                    file.write(line + '\n')

                # Agregar los renglones faltantes
                if n_entries < 105:
                    start_idx = n_entries + 1
                    for idx in range(start_idx, 106):
                        file.write(f"{1:>10}{1:>10}{idx:>10}{0.0:10.1f}{8:>10}\n")

# Crear una instancia del paquete SSM utilizando la clase personalizada
custom_ssm = CustomMt3dSsm(mt, stress_period_data=ssm_data, mxss=215)

# Escribir el archivo con el formato personalizado
custom_ssm.write_file()

C:\Users\Guillermo\anaconda3\Lib\site-packages\flopy\mbase.py:659: UserWarning: Unit 34 of package SSM already in use.
  warn(
C:\Users\Guillermo\anaconda3\Lib\site-packages\flopy\mbase.py:668: UserWarning: Two packages of the same type, Replacing existing 'SSM' package.
  warn(


In [11]:
# isothm: Define el tipo de isoterma. Se ha establecido a 1, lo que indica un modelo de adsorción lineal.
# ireact: Indica el tipo de reacción en el modelo. Se ha establecido a 1, lo que sugiere reacciones de primer orden.
# irctop: No está directamente utilizado en el constructor mostrado, pero usualmente controla la lectura 
# de los coeficientes de reacción.
# igetsc: Establecido en 0, indica que no se obtendrán concentraciones escalares adicionales.
# ireaction: No se utiliza en el constructor mostrado, pero generalmente controlaría opciones de reacción adicionales si se incluye.
# rhob: Densidad de la masa del suelo, fijada en 0.25, usada en el cálculo de la adsorción.
# rc1=lambda1: Tasa de reacción de primer orden para la fase disuelta, establecida en 0.002.
# rc2=lambda2: Tasa de decaimiento en la capa del suelo, también establecida en 0.002.
# rct: crea una instancia de la clase Mt3dRct en FloPy para configurar el paquete de reacción química en un modelo MT3DMS.

isothm = 1 # Isoterma de adsorción lineal
ireact = 1 # Reacciones de primer orden
irctop = 2 # controla la lectura de los coeficientes de reacción.
igetsc = 0 # no se obtendrán concentraciones escalares adicionales
ireaction = 0 # controla opciones de reacción adicionales
rhob = 0.25 # Densidad de masa del suelo
rc1 = 0.002  # Tasa de reacción de primer orden para la fase disuelta
rc2 = 0.002  # Decay on Soil Layer

rct = flopy.mt3d.Mt3dRct(
    mt, isothm=isothm, ireact=ireact, igetsc=igetsc, rhob = rhob, rc1=rc1, rc2=rc2
)

In [12]:
mt.write_input()


In [13]:
#------------------------------------------------------------------------

In [14]:
#mt.run_model()